In [3]:
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

print("✅ Keys loaded:")
print(f"OpenAI API Key: {openai_api_key[:5]}***")
print(f"Tavily API Key: {tavily_api_key[:5]}***")

openai_client = OpenAI(api_key=openai_api_key)

✅ Keys loaded:
OpenAI API Key: sk-pr***
Tavily API Key: tvly-***


In [4]:
def print_markdown(text: str):
    display(Markdown(text))

In [5]:
import json  # Import the json module for handling JSON data
from typing_extensions import TypedDict  # Import TypedDict for type hinting
from agents import function_tool

In [6]:
# Define a TypedDict for the expected parameters for the Tavily search function
# A TypedDict is like a blueprint for a dictionary in Python
# It tells Python exactly what keys the dictionary should have and what type of values go with each key.

class TavilySearchParams(TypedDict):
    query: str         # The search query string
    max_results: int   # The maximum number of results to return

In [7]:
# Let's define a function that searches the web using the Tavily API and gives back a short summary of the top results.
# Decorate the function as a FunctionTool for OpenAI Agents SDK
@function_tool
def tavily_search(params: TavilySearchParams) -> str:
    """
    Calls the Tavily API and returns a string summary of top search results.

    Args:
        params (TavilySearchParams): Dictionary with 'query' (str) and 'max_results' (int).

    Returns:
        str: A formatted string summarizing the top search results, or an error message.
    """
    print("Start tavily_search")
    
    # The web address (endpoint) for sending search requests to Tavily ( Tavily API endpoint)
    url = "https://api.tavily.com/search" 

    # Tell the API that we’re sending JSON data
    headers = {"Content-Type": "application/json"}  

    # What we’re sending to the API:
    # Our secret API key (so Tavily knows it's us)
    # The search text (query)
    # How many results we want (defaults to 2 if not given)
    payload = {
        "api_key": tavily_api_key,  # Use the Tavily API key from environment
        "query": params["query"],   # The search query from params
        "max_results": params.get("max_results", 2),  # Use max_results from params, default to 2 if not provided
    }

    # Send the search request to Tavily (POST means we’re sending data)
    response = requests.post(url, json = payload, headers = headers) 

    # Check if the search worked (200 = OK)
    if response.status_code == 200:  # If the request was successful
        results = response.json().get("results", [])  # Extract the 'results' list from the response JSON
        
        # Build a summary string with each result's title and content, numbered
        summary = "\n".join([f"{i+1}. {r['title']}: {r['content']}" for i, r in enumerate(results)])
        return summary if summary else "No relevant results found."  # Return summary or fallback message
    else:
        return f"Tavily API error: {response.status_code}"  # Return error message with status code if request failed

In [8]:
import json  # Import the json module for handling JSON data
from typing_extensions import TypedDict, Any  # Import TypedDict for type hinting, Any for general typing

# from agents import FunctionTool, function_tool, RunContextWrapper, CodeInterpreterTool # Import agent tool decorators and helpers
from agents import FunctionTool, function_tool, CodeInterpreterTool # Import agent tool decorators and helpers

# Use the hosted CodeInterpreterTool to provide a safe, sandboxed Python execution environment for the agent.
# This tool allows the agent to run Python code in a secure container, which is isolated from the main system.
# The 'tool_config' specifies that we want an automatic container (per documentation).
code_interpreter = CodeInterpreterTool(
    tool_config = {"type": "code_interpreter", 
                   "container": {"type": "auto"}})

# Print a message to confirm that all tools (get_catalog, run_readonly_sql, code_interpreter) are ready for use.
print("✅ Tool ready: code_interpreter")

✅ Tool ready: code_interpreter


In [13]:
from agents import Agent

analyst_agent = Agent(name = "Analyst Agent",
                      instructions = """
CONTEXT:
You are a world-class market research assistant with access to both real-time web search (via the tavily_search tool) and Python code execution (via the code_interpreter tool).

INSTRUCTION:
- Carefully analyze the user's question to determine whether it requires:
    - Recent or real-time information (use tavily_search),
    - Data analysis, calculations, or code execution (use code_interpreter),
    - Or a combination of both tools.
- Use tavily_search for up-to-date facts, news, or product information.
- Use code_interpreter for tasks involving data analysis, calculations, or code-based reasoning.
- If the task benefits from both tools, use them together and integrate the results.
- Clearly summarize your findings and reasoning. Do not copy-paste search results; always paraphrase.
- When using search, begin your answer with: "🔍 According to a web search …"
- When using code, explain your process and results clearly.

INPUT:
You will receive a conversation history and the latest user question. Use the full context to decide which tool(s) to use and to inform your response.

OUTPUT:
Provide a clear, well-structured answer. Reference search results and/or code outputs as appropriate, and integrate them into your summary.
""",
    model = "gpt-4.1-mini",
    tools = [tavily_search, code_interpreter],
)

print("✅ Agent created with Tavily and Code Interpreter tools.")

✅ Agent created with Tavily and Code Interpreter tools.


In [14]:
# Let's add memory to our agent
from agents import SQLiteSession
session = SQLiteSession("live_researcher_practice")

In [15]:
from agents import Runner
session = SQLiteSession("live_researcher")

# New question: Ask the agent to get the current price of a Tesla Cyber Truck in Canada,
# then simulate the price increase if tariffs rise from 5% to 20% in 1% increments,
# and calculate the new prices using Python, providing a summary across all scenarios.

q1 = (
    "Find the current price of a Tesla Cyber Truck in Canada. "
    "Then, simulate how the price would change if import tariffs increased from 5% to 20% in 1% increments. "
    "For each tariff rate, calculate the new price using Python and provide a summary of the results across all scenarios."
)

print_markdown(f"**User:** {q1}")

run1 = await Runner.run(
    starting_agent = analyst_agent,
    input = q1,
    session = session,
)

print_markdown(f"### 🤖 Agent’s Answer\n{run1.final_output}")

**User:** Find the current price of a Tesla Cyber Truck in Canada. Then, simulate how the price would change if import tariffs increased from 5% to 20% in 1% increments. For each tariff rate, calculate the new price using Python and provide a summary of the results across all scenarios.

Start tavily_search


### 🤖 Agent’s Answer
Here is a summary of the Tesla Cybertruck price changes in Canada with varying import tariff rates applied from 5% to 20%:

- Base price (no tariff): CAD 137,990
- Price at 5% tariff: CAD 144,889.50
- Price at 10% tariff: CAD 151,789.00
- Price at 15% tariff: CAD 158,688.50
- Price at 20% tariff: CAD 165,588.00

For each 1% increase in tariff from 5% to 20%, the price increases roughly by about CAD 1,380 (1% of base price).

Let me know if you want me to save this data or provide any additional analysis.